In [3]:
import MDAnalysis as md
import MDAnalysisTests
from MDAnalysis.tests.datafiles import PDB, XTC

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
from collections import Counter

from plip.structure.preparation import PDBComplex

import progressbar

import os

In [4]:
u = md.Universe('md_production.gro','md_production_ext_centered.xtc')

In [5]:
bar=progressbar.ProgressBar(max_value=len(u.trajectory))
table=pd.DataFrame()
index=0
for i in range(len(u.trajectory)):
    name='frame.pdb'
    PDB= md.Writer(name, multiframe=False)
    for ts in u.trajectory[i:i+1]:
        PDB.write(u.atoms)
        plip_job = PDBComplex()
        plip_job.load_pdb(name) 
        plip_job.analyze()
        interactions = plip_job.interaction_sets[list(plip_job.interaction_sets)[0]]
        for interaction in interactions.all_itypes:
            interaction_type=str(type(interaction)).split('.')[-1].replace("'>","")
            table.loc[index,'Frame']=ts.frame
            table.loc[index,'Time']=ts.time
            table.loc[index,'Type']=interaction_type
            table.loc[index,'Residue']=interaction.restype+str(interaction.resnr)
            table.loc[index,'Ligand']=interaction.restype_l+str(interaction.resnr_l)
            
            if interaction_type == 'hbond':
                table.loc[index,'Acceptor']=interaction.atype
                table.loc[index,'AcceptorIdx']=interaction.a.idx
                table.loc[index,'Donor']=interaction.dtype
                table.loc[index,'DonorIdx']=interaction.d.idx
                table.loc[index,'DistanceAD']=interaction.distance_ad
                table.loc[index,'DistanceAH']=interaction.distance_ah
                table.loc[index,'Angle']=interaction.angle
                table.loc[index,'Type']=interaction.type
                table.loc[index,'ProtIsDon']=interaction.protisdon
            
            elif interaction_type == 'pication':
                table.loc[index,'Charge']=interaction.charge.type
                table.loc[index,'ChargedAtoms']=",".join([i.type for i in interaction.charge.atoms])
                table.loc[index,'Type']=interaction.inter.type
                table.loc[index,'RingType']=interaction.ring.type
                table.loc[index,'RingAtoms']=",".join([i.type for i in interaction.ring.atoms])
                table.loc[index,'RingAtomsIdx']=",".join([str(i.idx) for i in interaction.ring.atoms])        
            
            elif interaction_type=='saltbridge':
                table.loc[index,'NegAtoms']=",".join([i.type for i in interaction.negative.atoms])
                table.loc[index,'NegAtomsIdx']=",".join([str(i.idx) for i in interaction.negative.atoms])
                table.loc[index,'PosAtoms']=",".join([i.type for i in interaction.positive.atoms])
                table.loc[index,'PosAtomsIdx']=",".join([str(i.idx) for i in interaction.positive.atoms])
                table.loc[index,'FunctGroup']=interaction.negative.fgroup
                table.loc[index,'Distance']=interaction.distance
                table.loc[index,'ProtIsPos']=interaction.protispos
                
            elif interaction_type == 'hydroph_interaction':
                table.loc[index,'RecAtom']=interaction.bsatom.type
                table.loc[index,'RecAtomIdx']=interaction.bsatom.idx
                table.loc[index,'LigAtom']=interaction.ligatom.type
                table.loc[index,'LigAtomIdx']=interaction.ligatom.idx
                table.loc[index,'Distance']=interaction.distance
                
            elif interaction_type == 'pistack':
                table.loc[index,'RecRingType']=interaction.proteinring.type
                table.loc[index,'LigRingType']=interaction.ligandring.type
                table.loc[index,'RecRingAtoms']=",".join([i.type for i in interaction.proteinring.atoms])
                table.loc[index,'RecAtomsIdx']=",".join([str(i.idx) for i in interaction.proteinring.atoms])
                table.loc[index,'LigRingAtoms']=",".join([i.type for i in interaction.ligandring.atoms])
                table.loc[index,'LigRingAtomsIdx']=",".join([str(i.idx) for i in interaction.ligandring.atoms])
                table.loc[index,'Distance']=interaction.distance
                table.loc[index,'Angle']=interaction.angle
                table.loc[index,'Offset']=interaction.offset
                
            elif interaction_type == 'waterbridge':
                table.loc[index,'AccType']=interaction.atype
                table.loc[index,'DonType']=interaction.dtype
                table.loc[index,'WaterIdx']=interaction.water_orig_idx
                table.loc[index,'DistanceAWat']=interaction.distance_aw
                table.loc[index,'DistanceDWat']=interaction.distance_dw
                table.loc[index,'AngleDon']=interaction.d_angle
                table.loc[index,'AngleWat']=interaction.w_angle
                table.loc[index,'ProtIsDon']=interaction.protisdon
  
            elif interaction_type=='metal_complex':
                table.loc[index,'MetalType']=interaction.metal.type
                table.loc[index,'Idx']=interaction.metal.idx
                table.loc[index,'TargetType']=interaction.target_type
                table.loc[index,'FunctGroup']=interaction.target.fgroup
                table.loc[index,'Geometry']=interaction.geometry
                table.loc[index,'Distance']=interaction.distance
                table.loc[index,'Location']=interaction.location
            
            index=index+1    
    bar.update(i)
    os.remove(name)
    
    
table.set_index(['Frame','Time','Type'], inplace=True)
table.sort_index(inplace=True)
table.to_excel('Interactions_Table.xlsx')

/mnt/c/Users/angel/Linux_programs/miniconda3/envs/AnalysisMD/lib/python3.7/site-packages/MDAnalysis/coordinates/PDB.py:916: UserWarning: Found no information for attr: 'altLocs' Using default value of ' '
  "".format(attrname, default))
/mnt/c/Users/angel/Linux_programs/miniconda3/envs/AnalysisMD/lib/python3.7/site-packages/MDAnalysis/coordinates/PDB.py:916: UserWarning: Found no information for attr: 'icodes' Using default value of ' '
  "".format(attrname, default))
/mnt/c/Users/angel/Linux_programs/miniconda3/envs/AnalysisMD/lib/python3.7/site-packages/MDAnalysis/coordinates/PDB.py:916: UserWarning: Found no information for attr: 'occupancies' Using default value of '1.0'
  "".format(attrname, default))
/mnt/c/Users/angel/Linux_programs/miniconda3/envs/AnalysisMD/lib/python3.7/site-packages/MDAnalysis/coordinates/PDB.py:916: UserWarning: Found no information for attr: 'tempfactors' Using default value of '0.0'
  "".format(attrname, default))
  0% (3 of 3209) |                       

KeyboardInterrupt: 

In [6]:
table.set_index(['Frame','Time','Type'], inplace=True)
table.sort_index(inplace=True)
#table.to_excel('Interactions_Table.xlsx')

In [8]:
table

Residue  Ligand RecAtom  RecAtomIdx LigAtom  \
Frame Time    Type                                                              
0.0   20000.0 hydroph_interaction    THR8  LIG152      C3        61.0     Car   
              hydroph_interaction   TYR11  LIG152     Car        86.0     Car   
              hydroph_interaction   TYR11  LIG152     Car        90.0     Car   
              hydroph_interaction   PHE55  LIG152     Car       427.0     Car   
              hydroph_interaction    ASN6  LIG152      C3        44.0     Car   
1.0   20010.0 hydroph_interaction   TYR11  LIG152     Car        89.0     Car   
              hydroph_interaction   TYR11  LIG152     Car        86.0     Car   
              hydroph_interaction  VAL130  LIG152      C3       996.0     Car   
              hydroph_interaction    ASN6  LIG152      C3        44.0     Car   
              hydroph_interaction   TYR11  LIG152     Car        90.0     Car   
              hydroph_interaction   PHE55  LIG152     Car       427.0     Car   
2.0   20020.0 hydroph_interaction   TYR11  LIG152     Car        86.0     Car   
              hydroph_interaction   PHE55  LIG152     Car       426.0     Car   
              hydroph_interaction    ASN6  LIG152      C3        44.0     Car   
              hydroph_interaction   TYR11  LIG152     Car        90.0     Car   
3.0   20030.0 hydroph_interaction   PHE55  LIG152     Car       426.0     Car   
              hydroph_interaction    ASN6  LIG152      C3        44.0     Car   
              hydroph_interaction   TYR11  LIG152     Car        90.0     Car   
              hydroph_interaction   THR57  LIG152      C3       443.0     Car   

                                   LigAtomIdx  Distance  
Frame Time    Type                                       
0.0   20000.0 hydroph_interaction         5.0  3.794338  
              hydroph_interaction         9.0  3.211760  
              hydroph_interaction        10.0  3.827284  
              hydroph_interaction         6.0  3.553562  
              hydroph_interaction        22.0  3.468616  
1.0   20010.0 hydroph_interaction         9.0  3.329955  
              hydroph_interaction        14.0  3.683083  
              hydroph_interaction        22.0  3.957790  
              hydroph_interaction        22.0  3.518878  
              hydroph_interaction        10.0  3.302575  
              hydroph_interaction         6.0  3.376107  
2.0   20020.0 hydroph_interaction        14.0  3.700622  
              hydroph_interaction         6.0  3.609335  
              hydroph_interaction        23.0  3.650329  
              hydroph_interaction         9.0  3.384583  
3.0   20030.0 hydroph_interaction         6.0  3.785499  
              hydroph_interaction        22.0  3.530269  
              hydroph_interaction         9.0  3.654668  
              hydroph_interaction         5.0  3.818809

In [ ]:
a= Counter(table.index.get_level_values('Frame'))

In [ ]:
figure= plt.figure(figsize=(10,10))
plt.tick_params ('both',width=2,labelsize=12)

plt.scatter(list(a.keys()),list(a.values()))

plt.title ('Non-Covalent Interactions',fontsize=14,fontweight='bold')
plt.xlabel ('Frame',fontsize=12,fontweight='bold')
plt.ylabel ('Number of Interactions',fontsize=12,fontweight='bold')

plt.tight_layout()
plt.savefig('NumInteraction.png',dpi=600,format='png',transparent=False)
plt.show()